In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
from collections import defaultdict
import pickle
from random import sample,random
import copy
from BOAmodel_JMLRv2 import *
from pylab import *
from random import sample
from sklearn.metrics import auc
%matplotlib inline
rcParams['figure.figsize'] = (10, 10)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def binary_code(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[col].quantile(float(q)/Nlevel)
            df[col+'_geq_'+str(int(q))+'q'] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)

# Generating Simulated Datasets

In [10]:
Nlevel = 5
Ntrue = 5
for N,J in zip([10000],[50]):
    for iteration in range(100):
        experiment = {}
        print('================== iter = {}, Ntrue = {} =================='.format(iteration,Ntrue))
        df = pd.DataFrame(30*np.random.rand(N,J),columns = ['att'+str(i+1) for i in range(J)])  
        df0 = df.copy(deep = True)
        df00 = df.copy(deep = True)
        binary_code(df,df.columns,Nlevel)
        train_index = sample(range(N),int(0.7*N))
        test_index = list(set(range(N)) - set(train_index))
        T = np.random.choice([0, 1], size=(len(train_index),), p=[0.5, 0.5])

#=== true set sugroup ====
        # generate true rule set
        flag = True
        while flag:
            w0 = np.random.rand(J) - 0.5
            g0 = np.dot(df0,w0)
            Y0 = (1.0/(1+np.exp(-g0))>=0.5).astype(int)
    #         print 'Mean of Y0 is {}'.format(mean(Y0))
            if mean(Y0)>=0.3 and mean(Y0)<=0.7:
                flag = False
        BOA_model = BOA(df.iloc[train_index],1-Y0[train_index])
        BOA_model.generate_rules(5,3,2000,True,5,'randomforest','precision')
        candidates = []
        threshold = sum(BOA_model.Y)*0.05
        for r in range(BOA_model.RMatrix.shape[1]):
            Z = np.nonzero(BOA_model.RMatrix[:,r])[0]
            if mean(BOA_model.Y[Z])>0.5 and sum(BOA_model.RMatrix[:,r])>threshold:
                candidates.append(r)
        TrueSet = [BOA_model.rules[i] for i in np.random.choice(candidates,Ntrue)]
        print(TrueSet)


        dfn = 1-df.iloc[train_index]
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df_train = pd.concat([df.iloc[train_index],dfn],axis = 1)
        Z = np.zeros(len(train_index))
        for rule in TrueSet:
            Z += (np.sum(df_train[rule],axis=1)==len(rule)).astype(int)
        Z = np.array((Z>0).astype(int))

# === inserting random noise ===
#         index = sample(range(len(train_index)),int(0.1*len(train_index)))[0]
#         Z[index] = 1-Z[index]
# #=== linear sugroup ====
#         flag = True
#         while flag:
#             ws = np.random.rand(J) - 0.5
#             gs = np.dot(df0,ws)
#             Z = (1.0/(1+np.exp(-gs))>=0.5).astype(int)
#     #         print 'Mean of Y0 is {}'.format(mean(Y0))
#             if mean(Z)>=0.2 and mean(Z)<=0.6:
#                 flag = False
#=== nonelinear sugroup ====
#         flag = True
#         while flag:
#             for k in range(10):
#                 if random()>0.5:
#                     col = sample(df00.columns.tolist(),1)
#                     df00['new'+str(k+1)] = pow(df00[col],sample([2,3],1)[0])
#                 else:
#                     cols = sample(df00.columns.tolist(),2)
#                     df00['new'+str(k+1)] = df00[cols[0]]*df00[cols[1]]
        
#             ws = np.random.rand(df00.shape[1]) - 0.5
#             gs = np.dot(df00,ws)
#             Z = (1.0/(1+np.exp(-gs))>=0.7).astype(int)
#     #         print 'Mean of Y0 is {}'.format(mean(Y0))
#             if mean(Z)>=0.1 and mean(Z)<=0.5:
#                 flag = False
## ==================================================
                
        ind = np.where(np.multiply(T,Z)==1)
        Y = Y0[train_index][:]
        Y[ind]=1

        dfn = 1-df.iloc[test_index]
        dfn.columns = [name.strip() + 'neg' for name in df.columns]
        df_test = pd.concat([df.iloc[test_index],dfn],axis = 1)

        Z_true = np.zeros(len(test_index))

        for rule in TrueSet:
            Z_true += (np.sum(df_test[rule],axis=1)==len(rule)).astype(int)
        Z_true = (Z_true>0).astype(int)



        experiment['Z_true'] = Z_true #Z[test_index] # 
        experiment['df'] = df_test #testing data
        experiment['test_index'] = test_index
        experiment['Y0'] = Y0
        experiment['T'] = T
        pickle.dump(experiment,open('simulations/'+str(iteration)+'.pkl','wb'))


================== iter = 0, Ntrue = 5 ==================
NEW -- Using random forest to generate rules ...
	Took 8.208s to generate 13607 rules
[['att12_geq_3q', 'att1_geq_2q', 'att24_geq_2qneg'], ['att1_geq_1qneg', 'att46_geq_3qneg', 'att6_geq_2q'], ['att18_geq_2qneg', 'att20_geq_2q', 'att44_geq_3qneg'], ['att12_geq_3q', 'att4_geq_4qneg', 'att6_geq_2q'], ['att29_geq_3qneg', 'att44_geq_2qneg', 'att6_geq_4q']]
================== iter = 1, Ntrue = 5 ==================
NEW -- Using random forest to generate rules ...


KeyboardInterrupt: 